In [1]:
using Distributions

function ci_wald(n, k; α=0.05)
    c = cquantile(Normal(), α/2)
    p̂ = k/n
    sehat = √(p̂*(1-p̂)/n)
    p̂ - c*sehat, p̂ + c*sehat
end

"""
kは確率1-qで二項分布Binomial(n, p0)に従う乱数であり,
確率qで二項分布Binomial(n, p1)に従う乱数であるとする.
n, kからWaldの100(1-α)%信頼区間[L, U]を計算する.
以上をniters回実行する.

条件A, Bの定義:

* A = (L ≤ p0 ≤ U)
* B = (L ≤ p1 ≤ U)

kが二項分布Binomial(n, p0)に従う乱数の場合について

* a = AかつBとなった回数
* b = Aかつnot Bとなった回数
* c = not AかつBとなった回数
* d = not Aかつnot Bとなった回数

kが二項分布Binomial(n, p1)に従う乱数の場合について

* e = AかつBとなった回数
* f = Aかつnot Bとなった回数
* g = not AかつBとなった回数
* h = not Aかつnot Bとなった回数

このとき

* g/(c+d+g+h)はp0が信頼区間に含まれない場合にkの生成に使われたp=p0, p1が信頼区間に含まれる条件付き確率の近似値になる.
* (a+b+e)/(a+b+e+f)はp0が信頼区間に含まれる場合にkの生成に使われたp=p0, p1が信頼区間に含まれる条件付き確率の近似値になる.
"""
function sim(; p0=0.5, p1=0.6, n=200, q=1e-4, α=0.05, niters=10^8)
    a, b, c, d, e, f, g, h = 0, 0, 0, 0, 0, 0, 0, 0
    for i in 1:niters
        if rand() > q
            k = rand(Binomial(n, p0))
            L, U = ci_wald(n, k; α)
            A = L ≤ p0 ≤ U
            B = L ≤ p1 ≤ U
            a += A & B
            b += A & !B
            c += !A & B
            d += !A & !B
        else
            k = rand(Binomial(n, p1))
            L, U = ci_wald(n, k; α)
            A = L ≤ p0 ≤ U
            B = L ≤ p1 ≤ U
            e += A & B
            f += A & !B
            g += !A & B
            h += !A & !B
        end
    end
    a, b, c, d, e, f, g, h
end

sim

In [2]:
@doc sim

kは確率1-qで二項分布Binomial(n, p0)に従う乱数であり, 確率qで二項分布Binomial(n, p1)に従う乱数であるとする. n, kからWaldの100(1-α)%信頼区間[L, U]を計算する. 以上をniters回実行する.

条件A, Bの定義:

  * A = (L ≤ p0 ≤ U)
  * B = (L ≤ p1 ≤ U)

kが二項分布Binomial(n, p0)に従う乱数の場合について

  * a = AかつBとなった回数
  * b = Aかつnot Bとなった回数
  * c = not AかつBとなった回数
  * d = not Aかつnot Bとなった回数

kが二項分布Binomial(n, p1)に従う乱数の場合について

  * e = AかつBとなった回数
  * f = Aかつnot Bとなった回数
  * g = not AかつBとなった回数
  * h = not Aかつnot Bとなった回数

このとき

  * g/(c+d+g+h)はp0が信頼区間に含まれない場合にkの生成に使われたp=p0, p1が信頼区間に含まれる条件付き確率の近似値になる.
  * (a+b+e)/(a+b+e+f)はp0が信頼区間に含まれる場合にkの生成に使われたp=p0, p1が信頼区間に含まれる条件付き確率の近似値になる.


In [3]:
@show a, b, c, d, e, f, g, h = sim(p0=0.5, p1=0.6, n=200, q=1e-4, α=0.05)
@show g/(c+d+g+h) (a+b+e)/(a+b+e+f) (a+b)/(a+b+c+d) (g+h)/(e+f+g+h);

(a, b, c, d, e, f, g, h) = sim(p0 = 0.5, p1 = 0.6, n = 200, q = 0.0001, α = 0.05) = (15100070, 79292053, 2797810, 2800129, 1471, 260, 7963, 244)
g / (c + d + g + h) = 0.001420405390797885
(a + b + e) / (a + b + e + f) = 0.999997245583383
(a + b) / (a + b + c + d) = 0.9440150462152929
(g + h) / (e + f + g + h) = 0.8258200845240491


In [4]:
@show a, b, c, d, e, f, g, h = sim(p0=0.5, p1=0.6, n=200, q=0.001, α=0.05)
@show g/(c+d+g+h) (a+b+e)/(a+b+e+f) (a+b)/(a+b+c+d) (g+h)/(e+f+g+h);

(a, b, c, d, e, f, g, h) = sim(p0 = 0.5, p1 = 0.6, n = 200, q = 0.001, α = 0.05) = (15089915, 79214073, 2797630, 2798744, 14527, 2703, 79973, 2435)
g / (c + d + g + h) = 0.014082773383447366
(a + b + e) / (a + b + e + f) = 0.9999713426092526
(a + b) / (a + b + c + d) = 0.9439804432340295
(g + h) / (e + f + g + h) = 0.8270740079086293


In [5]:
@show a, b, c, d, e, f, g, h = sim(p0=0.5, p1=0.6, n=200, q=0.01, α=0.05)
@show g/(c+d+g+h) (a+b+e)/(a+b+e+f) (a+b)/(a+b+c+d) (g+h)/(e+f+g+h);

(a, b, c, d, e, f, g, h) = sim(p0 = 0.5, p1 = 0.6, n = 200, q = 0.01, α = 0.05) = (14950640, 78505827, 2770631, 2771777, 147817, 26131, 802360, 24817)
g / (c + d + g + h) = 0.125967390340187
(a + b + e) / (a + b + e + f) = 0.9997209133378294
(a + b) / (a + b + c + d) = 0.9440154446199515
(g + h) / (e + f + g + h) = 0.8262474715944562


In [6]:
@show a, b, c, d, e, f, g, h = sim(p0=0.5, p1=0.6, n=300, q=0.01, α=0.01)
@show g/(c+d+g+h) (a+b+e)/(a+b+e+f) (a+b)/(a+b+c+d) (g+h)/(e+f+g+h);

(a, b, c, d, e, f, g, h) = sim(p0 = 0.5, p1 = 0.6, n = 300, q = 0.01, α = 0.01) = (18668730, 79412487, 459706, 459406, 184021, 4294, 803983, 7373)
g / (c + d + g + h) = 0.46460437292108264
(a + b + e) / (a + b + e + f) = 0.9999563038521441
(a + b) / (a + b + c + d) = 0.9907160712566925
(g + h) / (e + f + g + h) = 0.8116230239748877


In [7]:
@show a, b, c, d, e, f, g, h = sim(p0=0.5, p1=0.6, n=420, q=0.01, α=0.001)
@show g/(c+d+g+h) (a+b+e)/(a+b+e+f) (a+b)/(a+b+c+d) (g+h)/(e+f+g+h);

(a, b, c, d, e, f, g, h) = sim(p0 = 0.5, p1 = 0.6, n = 420, q = 0.01, α = 0.001) = (20089035, 78808304, 52195, 51952, 197670, 449, 799644, 751)
g / (c + d + g + h) = 0.8840319189158712
(a + b + e) / (a + b + e + f) = 0.9999954690153408
(a + b) / (a + b + c + d) = 0.9989480258912478
(g + h) / (e + f + g + h) = 0.8015861570293457


In [8]:
@show a, b, c, d, e, f, g, h = sim(p0=0.5, p1=0.6, n=480, q=0.01, α=0.0004)
@show g/(c+d+g+h) (a+b+e)/(a+b+e+f) (a+b)/(a+b+c+d) (g+h)/(e+f+g+h);

(a, b, c, d, e, f, g, h) = sim(p0 = 0.5, p1 = 0.6, n = 480, q = 0.01, α = 0.0004) = (19078195, 79880606, 21260, 21217, 187347, 187, 810896, 292)
g / (c + d + g + h) = 0.9498995507605442
(a + b + e) / (a + b + e + f) = 0.999998113899016
(a + b) / (a + b + c + d) = 0.9995709449326503
(g + h) / (e + f + g + h) = 0.8122260248597708


In [9]:
@show a, b, c, d, e, f, g, h = sim(p0=0.5, p1=0.6, n=200, q=0.5, α=0.05)
@show g/(c+d+g+h) (a+b+e)/(a+b+e+f) (a+b)/(a+b+c+d) (g+h)/(e+f+g+h);

(a, b, c, d, e, f, g, h) = sim(p0 = 0.5, p1 = 0.6, n = 200, q = 0.5, α = 0.05) = (7552247, 39653695, 1400463, 1399630, 7380286, 1315053, 40064927, 1233699)
g / (c + d + g + h) = 0.9085281366109523
(a + b + e) / (a + b + e + f) = 0.9764754406969672
(a + b) / (a + b + c + d) = 0.9440048986087379
(g + h) / (e + f + g + h) = 0.826072226917789


In [10]:
@show a, b, c, d, e, f, g, h = sim(p0=0.5, p1=0.6, n=200, q=0.8, α=0.05)
@show g/(c+d+g+h) (a+b+e)/(a+b+e+f) (a+b)/(a+b+c+d) (g+h)/(e+f+g+h);

(a, b, c, d, e, f, g, h) = sim(p0 = 0.5, p1 = 0.6, n = 200, q = 0.8, α = 0.05) = (3021135, 15860825, 559491, 560631, 11806843, 2106724, 64106523, 1977828)
g / (c + d + g + h) = 0.9539026219281564
(a + b + e) / (a + b + e + f) = 0.9357618494741676
(a + b) / (a + b + c + d) = 0.9439997296281457
(g + h) / (e + f + g + h) = 0.8260758861249364
